# Import nécessaires 


In [1]:
import pandas as pd
import csv
import os
import gc
from sklearn.preprocessing import StandardScaler
import sklearn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
import mlflow
import mlflow.sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix
import numpy as np

In [2]:
# Fonction pour fractionner et nettoyer un fichier CSV par morceaux
def process_csv_in_chunks(file_path, chunk_size=10000, expected_columns=122, sample_fraction=0.10):
    base_name = os.path.splitext(file_path)[0]
    chunk_number = 0
    cleaned_samples = []
    
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)
        chunk = [header]
        
        for i, line in enumerate(reader):
            if (i + 1) % chunk_size == 0:
                chunk.append(line)
                cleaned_chunk = clean_chunk(chunk, expected_columns)
                sample_chunk = sample_chunk_data(cleaned_chunk, sample_fraction)
                cleaned_samples.append(sample_chunk)
                chunk_number += 1
                chunk = [header]
            else:
                chunk.append(line)
        
        # Process remaining lines
        if chunk:
            cleaned_chunk = clean_chunk(chunk, expected_columns)
            sample_chunk = sample_chunk_data(cleaned_chunk, sample_fraction)
            cleaned_samples.append(sample_chunk)
    
    # Combine all cleaned samples into a single DataFrame
    final_sample = pd.concat(cleaned_samples, axis=0)
    return final_sample

In [3]:
# Fonction pour nettoyer un morceau de données
def clean_chunk(chunk, expected_columns):
    cleaned_chunk = [chunk[0]]  # Ajouter l'en-tête
    for row in chunk[1:]:
        if len(row) == expected_columns:
            cleaned_chunk.append(row)
    return cleaned_chunk

In [4]:
# Fonction pour échantillonner un morceau de données
def sample_chunk_data(chunk, sample_fraction):
    df_chunk = pd.DataFrame(chunk[1:], columns=chunk[0])
    sample_chunk = df_chunk.sample(frac=sample_fraction, random_state=1)
    return sample_chunk

In [5]:
# Liste des fichiers et leurs colonnes attendues
files_and_columns = {
    './application_train.csv': 122,
    './application_test.csv': 121,
}

# Processer chaque fichier CSV et sauvegarder les fragments propres
cleaned_samples = {}
for file, col_count in files_and_columns.items():
    print(f"Processing {file}...")
    cleaned_samples[file] = process_csv_in_chunks(file, chunk_size=10000, expected_columns=col_count, sample_fraction=0.10)
    print(f"Finished processing {file}.")
    print(cleaned_samples[file].head())
    cleaned_samples[file].to_csv(f'cleaned_{os.path.basename(file)}', index=False)  # Sauvegarder les fragments propres
    gc.collect()  # Nettoyer la mémoire après chaque fichier

Processing ./application_train.csv...


FileNotFoundError: [Errno 2] No such file or directory: './application_train.csv'

In [ ]:
# Fonction pour traiter les valeurs manquantes et encoder les variables catégorielles
def preprocess_data(df):
    # Séparation des colonnes numériques et catégorielles
    numeric_cols = df.select_dtypes(include=['number']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    # Imputation des valeurs manquantes pour les colonnes numériques uniquement
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
    
    # Encodage des variables catégorielles
    df = pd.get_dummies(df, columns=categorical_cols)
    
    return df

In [ ]:
# Fonction pour normaliser les données
def normalize_data(df):
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df_scaled

In [ ]:
# Charger les fichiers nettoyés
application_train = pd.read_csv('cleaned_application_train.csv')
application_test = pd.read_csv('cleaned_application_test.csv')

In [ ]:
application_train = preprocess_data(application_train)
application_test = preprocess_data(application_test)

In [ ]:
# Normaliser chaque fichier
application_train = normalize_data(application_train)
# application_test = normalize_data(application_test)

In [ ]:
# Charger les fichiers nettoyés
application_train = pd.read_csv('cleaned_application_train.csv')

# Imputation des valeurs manquantes pour les colonnes numériques
numeric_cols = application_train.select_dtypes(include=['number']).columns
application_train[numeric_cols] = application_train[numeric_cols].fillna(application_train[numeric_cols].mean())

# Vérifier les types de colonnes
print("Types de colonnes avant la suppression des colonnes non numériques :")
print(application_train.dtypes)

# Suppression des colonnes non numériques
application_train = application_train.select_dtypes(include=[float, int])

# Vérifier les types de colonnes après suppression
print("Types de colonnes après la suppression des colonnes non numériques :")
print(application_train.dtypes)

# Séparer les caractéristiques et la cible
X = application_train.drop('TARGET', axis=1)
y = application_train['TARGET']

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Appliquer SMOTE à l'ensemble d'entraînement
smote = SMOTE(random_state=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)




In [ ]:
# Save the DataFrame to a CSV file
file_path = './data.csv'
application_train.to_csv(file_path, index=False)

In [ ]:
# Définir les paramètres pour la recherche par grille
param_grid_lr = {
    'solver': ['liblinear'],
    'C': [0.1],
    'max_iter': [1000 ]
}

# Définir les paramètres pour la recherche par grille
param_grid_rf = {
    'n_estimators': [1, 10],
    'max_depth': [ None],
    'min_samples_split': [2]
}

# Définir les paramètres pour la recherche par grille
param_grid_gb = {
    'n_estimators': [1, 10],
    'learning_rate': [0.1],
    'max_depth': [5]
}


In [ ]:
# Configuration de l'expérience MLFlow
mlflow.set_experiment("Model Tracking and Registry")

# Fonction pour enregistrer les métriques avec MLFlow
def log_metrics_with_business_cost(model_name, best_model, X_test, y_test, y_pred_proba, fn_cost=10, fp_cost=1):
    auc = roc_auc_score(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, (y_pred_proba >= 0.5).astype(int))
    precision = precision_score(y_test, (y_pred_proba >= 0.5).astype(int))
    recall = recall_score(y_test, (y_pred_proba >= 0.5).astype(int))
    conf_matrix = confusion_matrix(y_test, (y_pred_proba >= 0.5).astype(int))
    
    best_threshold, best_cost = find_best_threshold(y_test, y_pred_proba, fn_cost, fp_cost)
    business_cost, y_pred_optimized = calculate_business_cost(y_test, y_pred_proba, best_threshold, fn_cost, fp_cost)
    
    mlflow.log_param("Model", model_name)
    mlflow.log_param("Best Params", best_model.get_params())
    mlflow.log_param("Best Threshold", best_threshold)
    mlflow.log_metric("AUC", auc)
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("Business Cost", business_cost)
    mlflow.sklearn.log_model(best_model, model_name)
    
    print(f"\n=== {model_name} ===")
    print(f"AUC: {auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Business Cost: {business_cost}")
    print("Confusion Matrix:")
    print(conf_matrix)

def calculate_business_cost(y_true, y_pred_proba, threshold, fn_cost=10, fp_cost=1):
    y_pred = (y_pred_proba >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cost = fn * fn_cost + fp * fp_cost
    return cost, y_pred

def find_best_threshold(y_true, y_pred_proba, fn_cost=10, fp_cost=1):
    thresholds = np.arange(0.0, 1.0, 0.01)
    costs = []
    
    for threshold in thresholds:
        cost, _ = calculate_business_cost(y_true, y_pred_proba, threshold, fn_cost, fp_cost)
        costs.append((cost, threshold))
    
    best_cost, best_threshold = min(costs, key=lambda x: x[0])
    return best_threshold, best_cost


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Entraîner et enregistrer les modèles avec GridSearchCV
with mlflow.start_run(run_name="Logistic Regression") as run:
    lr = LogisticRegression(random_state=1)
    grid_lr = GridSearchCV(lr, param_grid_lr, cv=5, scoring='roc_auc')
    grid_lr.fit(X_train_smote, y_train_smote)
    best_lr = grid_lr.best_estimator_
    y_pred_proba_lr = best_lr.predict_proba(X_test)[:, 1]
    log_metrics_with_business_cost("Logistic Regression", best_lr, X_test, y_test, y_pred_proba_lr)
    lr_run_id = run.info.run_id  # Obtenir l'ID de run pour l'enregistrement du modèle


# Random Forest

In [ ]:
with mlflow.start_run(run_name="Random Forest") as run:
    rf = RandomForestClassifier(random_state=1)
    grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='roc_auc')
    grid_rf.fit(X_train_smote, y_train_smote)
    best_rf = grid_rf.best_estimator_
    y_pred_proba_rf = best_rf.predict_proba(X_test)[:, 1]
    log_metrics_with_business_cost("Random Forest", best_rf, X_test, y_test, y_pred_proba_rf)
    rf_run_id = run.info.run_id  # Obtenir l'ID de run pour l'enregistrement du modèle

# Gradient Boost

In [ ]:
with mlflow.start_run(run_name="Gradient Boosting") as run:
    gb = GradientBoostingClassifier(random_state=1)
    grid_gb = GridSearchCV(gb, param_grid_gb, cv=5, scoring='roc_auc')
    grid_gb.fit(X_train_smote, y_train_smote)
    best_gb = grid_gb.best_estimator_
    y_pred_proba_gb = best_gb.predict_proba(X_test)[:, 1]
    log_metrics_with_business_cost("Gradient Boosting", best_gb, X_test, y_test, y_pred_proba_gb)
    gb_run_id = run.info.run_id  # Obtenir l'ID de run pour l'enregistrement du modèle


In [ ]:
# Enregistrement des modèles dans le registry
mlflow.register_model(f"runs:/{lr_run_id}/model", "LogisticRegressionModel")
mlflow.register_model(f"runs:/{rf_run_id}/model", "RandomForestModel")
mlflow.register_model(f"runs:/{gb_run_id}/model", "GradientBoostingModel")

In [ ]:
import subprocess

# Lancer le serveur MLflow en arrière-plan
subprocess.Popen(["mlflow", "ui", "--port", "5000"])


In [ ]:
import IPython.display as display

# Afficher le lien pour accéder à l'interface utilisateur de MLflow
display.display(display.Javascript('window.open("//localhost:5000", "_blank");'))


In [ ]:
# !cd C:\Users\Foudil
# !git init


In [ ]:
# import os

# # Ajouter les fichiers à l'index
# os.system("git add P7_Analyse_exploratoire.ipynb cleaned_application_train.csv")

# # Faire un commit avec un message approprié
# os.system('git commit -m "Initial commit with analysis and data, ignoring large files"')


In [ ]:
# import os
# import subprocess

# # Fonction pour exécuter une commande et capturer la sortie et les erreurs
# def run_command(command):
#     result = subprocess.run(command, shell=True, capture_output=True, text=True)
#     print("Output:", result.stdout)
#     print("Error:", result.stderr)
#     return result.returncode

# # Retirer le fichier volumineux de l'index Git
# # run_command("git rm --cached datap7/application_train_cleaned.csv")

# # Ajouter .gitignore à l'index
# # run_command("git add .gitignore")

# # Faire un commit avec un message approprié
# run_command('git commit -m "Remove large file from version control"')


In [ ]:
# import os

# # Ajouter les fichiers à l'index
# os.system("git add P7_Analyse_exploratoire.ipynb cleaned_application_train.csv")

# # Faire un commit avec un message approprié
# os.system('git commit -m "Initial commit with analysis and data"')

# # Pousser les changements vers GitHub
# os.system("git push -u origin main")


In [ ]:
# name: CI

# on: [push]

# jobs:
#   build:
#     runs-on: ubuntu-latest

#     steps:
#       - uses: actions/checkout@v2
#       - name: Set up Python
#         uses: actions/setup-python@v2
#         with:
#           python-version: 3.x
#       - name: Install dependencies
#         run: |
#           python -m pip install --upgrade pip
#           pip install -r requirements.txt
#       - name: Run tests
#         run: |
#           pytest


http://localhost:5000 MLFlow UI

In [ ]:
cost_ratios = [(5, 1), (10, 1), (20, 1)]

def evaluate_cost_ratios(y_test, y_pred_proba, cost_ratios):
    results = []
    for fn_cost, fp_cost in cost_ratios:
        best_threshold, business_cost = find_best_threshold(y_test, y_pred_proba, fn_cost, fp_cost)
        results.append({
            'FN Cost': fn_cost,
            'FP Cost': fp_cost,
            'Best Threshold': best_threshold,
            'Business Cost': business_cost
        })
    return results


In [ ]:
results_lr = evaluate_cost_ratios(y_test, y_pred_proba_lr, cost_ratios)
print("\n=== Logistic Regression Costs ===")
print(pd.DataFrame(results_lr))


In [ ]:
results_rf = evaluate_cost_ratios(y_test, y_pred_proba_rf, cost_ratios)
print("\n=== Random Forest Costs ===")
print(pd.DataFrame(results_rf))


In [ ]:
results_gb = evaluate_cost_ratios(y_test, y_pred_proba_gb, cost_ratios)
print("\n=== Gradient Boosting Costs ===")
print(pd.DataFrame(results_gb))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def compare_models(cost_ratios, results_lr, results_rf, results_gb):
    comparison = []
    for i, (fn_cost, fp_cost) in enumerate(cost_ratios):
        comparison.append({
            'Model': 'Logistic Regression',
            'FN Cost': fn_cost,
            'FP Cost': fp_cost,
            'Best Threshold': results_lr[i]['Best Threshold'],
            'Business Cost': results_lr[i]['Business Cost']
        })
        comparison.append({
            'Model': 'Random Forest',
            'FN Cost': fn_cost,
            'FP Cost': fp_cost,
            'Best Threshold': results_rf[i]['Best Threshold'],
            'Business Cost': results_rf[i]['Business Cost']
        })
        comparison.append({
            'Model': 'Gradient Boosting',
            'FN Cost': fn_cost,
            'FP Cost': fp_cost,
            'Best Threshold': results_gb[i]['Best Threshold'],
            'Business Cost': results_gb[i]['Business Cost']
        })
    return pd.DataFrame(comparison)

comparison_df = compare_models(cost_ratios, results_lr, results_rf, results_gb)
print("\n=== Model Comparison with Different Cost Ratios ===")
print(comparison_df)







Business Cost by Model and FN Cost : Ce graphique montre le coût métier pour chaque modèle en fonction du coût des faux négatifs. On observe que le modèle Random Forest avec un coût FN de 5 a le coût métier le plus bas

In [ ]:
# Visualisation
plt.figure(figsize=(12, 8))
sns.barplot(data=comparison_df, x='Model', y='Business Cost', hue='FN Cost')
plt.title('Business Cost by Model and FN Cost')
plt.ylabel('Business Cost')
plt.xlabel('Model')
plt.legend(title='FN Cost')
plt.show()

Best Threshold by Model and FN Cost : Ce graphique montre le meilleur seuil pour chaque modèle en fonction du coût des faux négatifs. On observe que les seuils varient significativement en fonction du coût FN et du modèle.

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=comparison_df, x='Model', y='Best Threshold', hue='FN Cost')
plt.title('Best Threshold by Model and FN Cost')
plt.ylabel('Best Threshold')
plt.xlabel('Model')
plt.legend(title='FN Cost')
plt.show()

In [ ]:
import shap
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer

In [ ]:

# Créer l'objet explainer SHAP
explainer_lr = shap.Explainer(best_lr, X_train)


In [ ]:
# Calculer les valeurs SHAP pour l'ensemble de test
shap_values_lr = explainer_lr(X_test)


In [ ]:
# Initialisation de JavaScript pour les visualisations SHAP
shap.initjs()

# Visualisation des valeurs SHAP pour une prédiction individuelle
shap.force_plot(shap_values_lr[0])



In [ ]:
# Visualisation globale des valeurs SHAP
shap.summary_plot(shap_values_lr, X_test)


In [ ]:
# Extraire les valeurs SHAP
shap_values_array = shap_values_lr.values if hasattr(shap_values_lr, 'values') else shap_values_lr

# Extraire les indices des 10 caractéristiques les plus importantes
importances = np.abs(shap_values_array).mean(axis=0)
indices = np.argsort(importances)[-10:]  # Obtenir les indices des 10 caractéristiques les plus importantes
important_features = X_test.columns[indices]
print("Les 10 caractéristiques les plus importantes sont :")
print(important_features)

In [ ]:

# Créer l'objet explainer SHAP
explainer_rf = shap.Explainer(best_rf, X_train)


In [ ]:
# Initialiser les SHAP explainers pour chaque modèle avec check_additivity=False
explainers = {
    "Random Forest": shap.TreeExplainer(best_rf, check_additivity=False),
    "Gradient Boosting": shap.TreeExplainer(best_gb, check_additivity=False),
    "Logistic Regression": shap.LinearExplainer(best_lr, X_train, check_additivity=False)
}

In [ ]:
import pickle
import pandas as pd
import traceback

# Essayer de charger le modèle de scoring
try:
    with open('best_lr_important.pkl', 'rb') as f:
        model = pickle.load(f)
    print("Modèle chargé avec succès.")
except Exception as e:
    print("Erreur lors du chargement du modèle :", e)
    print(traceback.format_exc())

# Essayer de charger les données du fichier CSV
try:
    df = pd.read_csv('cleaned_application_test.csv')
    print("Données chargées avec succès.")
except Exception as e:
    print("Erreur lors du chargement des données :", e)
    print(traceback.format_exc())


In [ ]:
import os

# Obtenir le chemin absolu du répertoire courant
chemin_projet = os.getcwd()
print("Chemin du projet :", chemin_projet)

In [ ]:
import pickle

# Supposez que best_lr est votre modèle de régression logistique déjà entraîné
with open('best_lr.pkl', 'wb') as model_file:
    pickle.dump(best_lr, model_file)


In [ ]:
import requests

# Fonction pour faire une requête à l'API et obtenir des prédictions pour un client spécifique
def get_client_prediction(client_id, api_url):
    try:
        data = {"client_id": client_id}
        response = requests.post(api_url, json=data)
        print(f"Status code: {response.status_code}")  # Afficher le code de statut HTTP
        print(f"Response content: {response.content}")  # Afficher le contenu de la réponse
        if response.status_code == 200:
            return response.json()
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

# URL de l'API (à remplacer par l'URL réelle de votre API)
api_url = "https://foudilnini.pythonanywhere.com/predict"

# Sélectionner un client pour la prédiction
client_id = 243566# Remplacez par l'ID du client que vous souhaitez prédire

# Obtenir la prédiction pour le client sélectionné
prediction = get_client_prediction(client_id, api_url)

# Afficher la prédiction
if prediction:
    print(f"Prediction for client {client_id}: {prediction}")
else:
    print(f"Failed to get prediction for client {client_id}.")
